In [1]:
'''
现在的标题三元组和正文三元组全部提取完成，但是可以看到存在很多相似的三元组以及一些无用的三元组（比如实体是标点符号）

下面需要做的就是进行知识融合，知识融合主要分为实体对齐和实体消歧，但是因为实体消歧这部分在我们的任务中并不存在（专业领域很少有存在歧义的实体），因此只需要进行实体对齐即可

查询GPT得知还有什么关系规范化，这些步骤就太复杂了，我们就不再考虑
'''
import os
import numpy as np
import ast

from tqdm import tqdm
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

import synonyms # 中文近义词库

import csv
import random

Building prefix dict from d:\Download_software\My_miniconda\envs\pytorch_py3.7\lib\site-packages\synonyms\data\vocab.txt ...
Loading model from cache C:\Users\a7386\AppData\Local\Temp\jieba.u1fa9792105018516e1e68c84fecdf626.cache



 Synonyms: v3.18.0, Project home: https://github.com/chatopera/Synonyms/

 Project Sponsored by Chatopera

  deliver your chatbots with Chatopera Cloud Services --> https://bot.chatopera.com

>> Synonyms load wordseg dict [d:\Download_software\My_miniconda\envs\pytorch_py3.7\lib\site-packages\synonyms\data\vocab.txt] ... 


Loading model cost 1.305 seconds.
Prefix dict has been built successfully.


>> Synonyms on loading stopwords [d:\Download_software\My_miniconda\envs\pytorch_py3.7\lib\site-packages\synonyms\data\stopwords.txt] ...
>> Synonyms on loading vectors [d:\Download_software\My_miniconda\envs\pytorch_py3.7\lib\site-packages\synonyms\data\words.vector.gz] ...


在着手处理之前还是需要先看一下我们的三元组的形式，确定哪些三元组需要清理

·两个完全相同的三元组 ['东北局部地区', '遭受', '洪涝灾害'] 和 ['东北局部地区', '遭受', '洪涝灾害'] -- 去重

·头实体或尾实体是单个字符(包括标点符号)的三元组，如['已', '致', '85人死亡']、[',', '致', '85人死亡']、['18人', '死', '4'] -- 在中文中要表示一个实体几乎是不可能只用一个字的

·头实体或尾实体以“等”开头（这是我们文本的特殊性导致的一个问题），这种三元组不用删除，只需要删除头实体的“等”即可，如['等湘省份', '遭受', '洪涝灾害'] -- 规范化

·头实体或尾实体是以数字开头的（同样是文本特殊性导致），如['1.6亿', '投入', '5万人']，这种三元组不具备语义 -- 语义角度，灾情文本中两个数字实体之间几乎不会有什么有用的关系（头实体和尾实体都是数字开头），亦或头实体或尾实体以数字开头，往往并不是我们需要的“唯一”实体

·头实体或尾实体以“你”、“我”、“他”人称代词开头，不符合作为唯一实体的要求

·头实体等于尾实体，自己与自己存在关系，从常理上讲行不通

·头实体或尾实体意义接近，可以认为是两个相同的三元组如 ['两部门', '启动', '四级应急响应'] 和['两部门', '启动', 'Ⅳ级应急响应'] -- 可以用jaccard距离或余弦相似度来判断


In [ ]:
# 1.删除重复的三元组 -- 使用集合，因为集合只保留唯一的元素。
# 之所以不把重复三元组放在相似三元组中处理是为了减少要处理的数据量

triplets = [] 
with open('./data/triples_data.txt', 'r') as file:
    triplets = [line.strip() for line in file]

unique_triplets = set()

for triplet in triplets:
    unique_triplets.add(triplet)

unique_triplets = list(unique_triplets)

with open('./data/unique_triples_data.txt', 'w') as file:
    file.write('\n'.join(unique_triplets))

# print(unique_triplets)


In [ ]:
# 小知识点 -- 使用ast.literal_eval()函数将字符串转换为列表，提取三元组中的实体和关系
# import ast

# triplet_str = "['洪涝灾害', '造成', '10.7万人受灾']"

# # Safely evaluate the string as a literal and convert it into a list
# triplet_list = ast.literal_eval(triplet_str)

# # Extract the values from the triplet list
# head_entity = triplet_list[0]
# relationship = triplet_list[1]
# tail_entity = triplet_list[2]

# # Print the extracted values
# print("Head Entity:", head_entity)
# print("Relationship:", relationship)
# print("Tail Entity:", tail_entity)


In [ ]:
# 2.删除其他不符合条件的三元组


with open('./data/unique_triples_data.txt', 'r') as file_in,open('./data/clear_triples_data.txt', 'w') as file_out:
    for line in file_in:
        triplet_str = line.strip()
        # print(triplet_str)
        triplet_list = ast.literal_eval(triplet_str)
        head_entity = triplet_list[0]
        relationship = triplet_list[1]
        tail_entity = triplet_list[2]
        if not (len(head_entity) ==1 or len(tail_entity) ==1 or head_entity[0].isdigit() or tail_entity[0].isdigit() or head_entity == tail_entity or \
                    head_entity[0]=="我" or tail_entity[0]=="我" or head_entity[0]=="你" or tail_entity[0]=="你" or head_entity[0]=="他" or tail_entity[0]=="他"):
            # print(triplet_list)
            if head_entity[0] == "等":
                head_entity = head_entity.replace("等","")
            if tail_entity[0] == "等":
                tail_entity = tail_entity.replace("等","")
            # print(head_entity,relationship,tail_entity)
            file_out.write(str(head_entity)+'\t'+str(relationship)+'\t'+str(tail_entity)+'\n')
        



额外的，还需要限定关系的类型数量以及实体的类型数量（这一步应该在计算整个三元组相似性之前） -- 而如果想要基于语料库中已有的实体标签取其子集作为实体，这种效果不会很好，原因是语料中的实体如“人”、“房屋”、“农作物”，这还仅仅只是承载体，如果要算上人口面积（诸如“死亡失踪101人/ADP”，“转移安置18.9万人次/ATP”）、时间标签（如“7月份/DS”，“7月21日/DO”）等，因为本身的标注实体的效果就不好，如果再和我们使用依存分析得到的结果进行一个“负负得正”的操作，最后得到的效果会很差。同时，也不能为了减少实体就刻意的去直接删除某些实体，这是不负责任的表现，比如“江西省”和“江西减灾委”就是两个实体，不能将其作为同一个实体。对于关系也是如此，如果仅仅只想限制关系的数量仅限于“发生在”、“有”这种，不仅不现实，而且会极大的减少三元组的数量。不能从集合的角度来处理实体和关系的话，可以从词嵌入即更高层次的技术来处理。

因为实体和关系现在都可以看作是单独的中文词语，因此处理词语特别是同义词、近义词，比如“出现”、“出现在”、“出现于”这三种关系实际上都是同一种关系，将其融合以减少关系的数量 -- 可以使用一种被称为同义词规范化的技术，选择使用最短的同义词或使用频率最频繁的同义词（随机也OK）作为规范形式。

注意因为这里需要使用到词嵌入，之所以后面的三元组没有使用词嵌入是因为这两个步骤处理的视角不同。此处需要考虑为词嵌入是因为考虑到语义方面的不同（这需要深层次的对语言的理解，词嵌入是一个不错的选择）；而清理三元组是仅仅只需要将汉字看作是普通的集合中的元素，只需要比对集合中元素的个数即可。

回到正文，我们确定任务为寻找并清理中文近义词、同义词，要么使用现成的库（可能在专业邻域的效果不是特别好），要么自己训练word2vec（利用专业邻域文本）



In [7]:
# # 示例：synonyms计算两个词语的相似度（基于自己训练得到的word2Vec，可以看到效果好得多，并且针对没有出现过的词也不会报错）
# word1 = "Ⅰ级"
# word2 = "Ⅲ级"
# r = synonyms.compare(word1, word2, seg=False)
# print(r)

0.852


In [ ]:
# # 下面的代码是基于csv文件的，实际上我们应该基于clean_triples_data.txt文件进行操作
# def replace_similar_words(word,unique_words,threshold=0.7):
#     if unique_words == []:
#         unique_words.append(word)
#         return word
#     else:
#         max_similarity = 0
#         unique_str = ''
#         for unique_word in unique_words:
#             similarity = synonyms.compare(word, unique_word, seg=False)
#             if similarity > max_similarity:
#                 max_similarity = similarity # 保存最大相似度
#                 unique_str = unique_word # 保存最大相似度对应的词
#         if max_similarity >= threshold:
#             return unique_str # 返回最大相似度对应的词
#         else:
#             unique_words.append(word)
#             return word
# # 读取csv文件第三行的关系词，进行替换
# csv_file_path = './data/roles.csv'

# with open(csv_file_path, 'r', encoding='utf-8') as file:
#     reader = csv.reader(file)
#     rows = list(reader)

#     unique_words = []
#     # replace_words = []

#     for row in rows:
#         if len(row) >= 3: # 确保关系csv文件有3列
#             word = row[2]  # 获取第三列的关系词
#             replaced_word = replace_similar_words(word,unique_words)
#             # print(replaced_word)
#             row[2] = replaced_word # 使用replaced_word替换原来的词
#             # replace_words.append(replaced_word) # 保存替换后的词，便于查看变量

#         else:
#             print('csv error...')

# with open(csv_file_path, 'w', encoding='utf-8', newline='') as file:
#     writer = csv.writer(file)
#     writer.writerows(rows)

# print("Roles replacement completed successfully!")

In [8]:
# 3.词嵌入分别计算实体和关系词相似性（即近义词、同义词匹配），替换相似度较高的词，以进行实体、关系的匹配
def replace_similar_words(word,unique_words,threshold=0.7):
    if unique_words == []:
        unique_words.append(word)
        return word
    else:
        max_similarity = 0
        unique_str = ''
        for unique_word in unique_words:
            similarity = synonyms.compare(word, unique_word, seg=False)
            if similarity > max_similarity:
                max_similarity = similarity # 保存最大相似度
                unique_str = unique_word # 保存最大相似度对应的词
        if max_similarity >= threshold:
            return unique_str # 返回最大相似度对应的词
        else:
            unique_words.append(word)
            return word

In [9]:
input_file_path = './data/clear_triples_data.txt'
output_file_path = './data/synonyms_triples_data.txt'

with open(input_file_path, 'r',encoding = 'utf-8') as file_in,open(output_file_path, 'w',encoding = 'utf-8') as file_out:
    lines = file_in.readlines()
    head1_replaced_words = [] # 保存实体1替换后的词
    head2_replaced_words = [] # 保存实体2替换后的词
    relation_replaced_words = [] # 保存关系替换后的词
    
    head1_unique_words = []
    for line in lines:
        words = line.strip().split('\t')
        # print(words[1]) # words[0]对应实体1，words[1]对应关系，words[2]对应实体2
        head1_replaced_word = replace_similar_words(words[0],head1_unique_words) # 替换实体1
        head1_replaced_words.append(head1_replaced_word)
    
    head2_unique_words = []
    for line in lines:
        words = line.strip().split('\t')
        head2_replaced_word = replace_similar_words(words[2],head2_unique_words)
        head2_replaced_words.append(head2_replaced_word)
    
    relation_unique_words = []
    for line in lines:
        words = line.strip().split('\t')
        relation_replaced_word = replace_similar_words(words[1],relation_unique_words)
        relation_replaced_words.append(relation_replaced_word)

    # 将替换后的词写入文件
    for i in range(len(lines)):
        file_out.write(head1_replaced_words[i] + '\t' + relation_replaced_words[i] + '\t' + head2_replaced_words[i] + '\n')
        


d:\Download_software\My_miniconda\envs\pytorch_py3.7\lib\site-packages\synonyms\utils.py:246: RuntimeWarning: invalid value encountered in true_divide
  cosine = lambda a, b: dot(a, b)/(norm(a)*norm(b))
d:\Download_software\My_miniconda\envs\pytorch_py3.7\lib\site-packages\synonyms\utils.py:246: RuntimeWarning: invalid value encountered in true_divide
  cosine = lambda a, b: dot(a, b)/(norm(a)*norm(b))
d:\Download_software\My_miniconda\envs\pytorch_py3.7\lib\site-packages\synonyms\utils.py:246: RuntimeWarning: invalid value encountered in true_divide
  cosine = lambda a, b: dot(a, b)/(norm(a)*norm(b))
d:\Download_software\My_miniconda\envs\pytorch_py3.7\lib\site-packages\synonyms\utils.py:246: RuntimeWarning: invalid value encountered in true_divide
  cosine = lambda a, b: dot(a, b)/(norm(a)*norm(b))
d:\Download_software\My_miniconda\envs\pytorch_py3.7\lib\site-packages\synonyms\utils.py:246: RuntimeWarning: invalid value encountered in true_divide
  cosine = lambda a, b: dot(a, b)/(no

In [ ]:
# '''
# jacard 算法参考：https://blog.51cto.com/u_9453611/5569493
# 这个算法主要是用来比较两个集合之间的相似性或重叠的相似性度量
# jacard算法通过两个集合的交集除以并集得到的，也就是说，如果两个集合完全相同，那么它们的jaccard系数就是1，如果两个集合完全不同，那么它们的jaccard系数就是0
# jacard算法本身和实体对齐是没有直接关系的，但是因为中文的特殊性（可以认为中文实体就是一个集合，其中的字符就是集合元素），中文字符重叠率越高则意义越可能相似
# -- 本来基于属性的实体对齐的方法有很多，但是因为我们的三元组是（实体，关系，实体），因此能够选择的方式就少得多
# '''
# def edit_distance(word1, word2):
#     len1 = len(word1)
#     len2 = len(word2)
#     dp = np.zeros((len1 + 1, len2 + 1))
#     for i in range(len1 + 1):
#         dp[i][0] = i
#     for j in range(len2 + 1):
#         dp[0][j] = j
#     for i in range(1, len1 + 1):
#         for j in range(1, len2 + 1):
#             delta = 0 if word1[i - 1] == word2[j - 1] else 1
#             dp[i][j] = min(dp[i - 1][j - 1] + delta, min(dp[i - 1][j] + 1, dp[i][j - 1] + 1))
#     return dp[len1][len2]
# def Jaccrad(terms_model,reference):
#     grams_reference = set(reference)
#     grams_model = set(terms_model)
#     temp = 0
#     for i in grams_reference:
#         if i in grams_model:
#             temp = temp + 1
#     fenmu = len(grams_model) + len(grams_reference) - temp
#     jaccard_coefficient = float(temp / fenmu)
#     return jaccard_coefficient

# blists=["四级应急响应","Ⅳ级应急响应",'四级救灾应急响应','国家四级应急响应']
# for i in range(len(blists)):
#     for j in range(0,i):
#         a = blists[i]
#         b = blists[j]
#         print(blists[i],blists[j])
#         td = Jaccrad(a, b)
# #         print(td)
#         std =edit_distance(a, b)/max(len(a),len(b))
#         fy = 1-std
# #         print(fy)
#         huizon = (td+fy)/2
#         print('avg_sim: ', huizon)

In [ ]:
# # 4.删除相似三元组（利用余弦相似度 -- 使用余弦相似度相比于直接使用jaccard准度更好，不至于删除过多的三元组（该部分因为是迭代进行，因此会处理非常久）
# # 需要明确的是，这一个步骤不管是放在实体和关系对齐之前还是之后，因为三元组的数量始终都是一样的，因此运行的都会耗费相同的大量时间（如果能够确保三元组的纯净性，则直接使用condition_triples_data也足够）
# # 这个步骤从原理上来说也应该是放在实体和关系对齐之后，因为它是基于集合的浅层计算
# # 这里因为这一步需要耗费大量时间，我们并没有执行
# triplets = []
# with open('./data/synonyms_triples_data.txt', 'r') as file:
#     triplets = [line.strip() for line in file]

# unique_and_similar_triplets = []

# # 将每个三元组与现有的唯一三元组进行比较
# for triplet in tqdm(triplets):
#     is_duplicate = False
#     for unique_triplet in unique_and_similar_triplets:
#         # 将三元组转换为TF-IDF向量
#         vectorizer = TfidfVectorizer()
#         tfidf_matrix = vectorizer.fit_transform([triplet, unique_triplet])

#         # 计算余弦相似度
#         similarity = cosine_similarity(tfidf_matrix[0], tfidf_matrix[1])[0][0]

#         # 相似性阈值（根据需要进行调整）
#         threshold = 0.8

#         if similarity >= threshold:
#             is_duplicate = True
#             break

#     # 跳过相似的三元组
#     if is_duplicate:
#         continue

#     # 添加唯一的三元组 
#     unique_and_similar_triplets.append(triplet)

# # 写入文件
# with open('./data/final_triplets_data.txt', 'w') as file:
#     file.write('\n'.join(unique_and_similar_triplets))


# print('finished...')

这里举个简单的例子解释上面代码的功能，假如我们的txt文件中的三元组内容如下：

apple, fruit, red
banana, fruit, yellow
apple, fruit, green
orange, fruit, orange
grape, fruit, purple
apple, fruit, red

经过下面代码的运行过后，会得到如下的结果

apple, fruit, red
banana, fruit, yellow
apple, fruit, green
orange, fruit, orange
grape, fruit, purple

其中的apple, fruit, red三元组因为计算出来的similarity超过了阈值0.8，因此会被删除

详细的流程为
1.先读入三元组并存储在triplets变量中
triplets = ['apple, fruit, red','banana, fruit, yellow','apple, fruit, green','orange, fruit, orange','grape, fruit, purple','apple, fruit, red']

2.初始化空列表unique_and_similar_triplets
unique_and_similar_triplets = []

3.接着，以此迭代和计算相似度即可（使用如下代码）

'''
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(['apple, fruit, red','banana, fruit, yellow'])
similarity = cosine_similarity(tfidf_matrix[0], tfidf_matrix[1])[0][0]
'''

第一轮因为'apple, fruit, red'和空unique_and_similar_triplets计算的结果为0小于阈值，因此直接将'apple, fruit, red'加入列表；
第二轮，'banana, fruit, yellow'和'apple, fruit, red'的相似度为0.2，因此将'banana, fruit, yellow'加入列表
第三轮，'apple, fruit, green'和列表中的'apple, fruit, red'和'banana, fruit, yellow'的相似度分别为0.5和0.2，因此将'apple, fruit, green'加入列表
...
最后'apple, fruit, red'与'apple, fruit, red'进行计算的时候会出现相似度为1的情况，因此将其丢弃

其中使用的TfidfVectorizer类是scikit-learn库中的一个类，该类专门将原始文档集合转换为TF-IDF特征矩阵。

In [ ]:
# # 示例：计算三元组的余弦相似度
# triplets = []
# with open('./data/clear_triples_data.txt', 'r') as file:
#     triplets = [line.strip() for line in file]

# vectorizer = TfidfVectorizer()
# tfidf_matrix = vectorizer.fit_transform([triplets[0],triplets[0]])
# similarity = cosine_similarity(tfidf_matrix[0], tfidf_matrix[1])[0][0]
# print(similarity)

最后，为了保证构成的知识图谱的效果足够好，我们可以手动再对实体或关系进行调整（毕竟机器的能力再强也不可能强过人，一定程度的人工清洗不仅可以提高三元组质量，为良好的可视化打下基础，也可以将这种优质的三元组用于训练机器模型）

完成上述步骤后，三元组的清洗工作基本结束